# Exploring Hacker News Posts

In this project, we'll work with a data set of submissions to popular technology site Hacker News.

Hacker News is a site started by the startup incubator Y Combinator, where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

You can find the data set on https://www.kaggle.com/hacker-news/hacker-news-posts, but note that it has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions.

We're specifically interested in posts whose titles begin with either Ask HN or Show HN. Users submit Ask HN posts to ask the Hacker News community a specific question. Below are a couple examples:

- *Ask HN: How to improve my personal website?*
- *Ask HN: Am I the only one outraged by Twitter shutting down share counts?*
- *Ask HN: Aby recent changes to CSS that broke mobile?*

Likewise, users submit Show HN posts to show the Hacker News community a project, product, or just generally something interesting. Below are a couple of examples:

- *Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform'*
- *Show HN: Something pointless I made*
- *Show HN: Shanhu.io, a programming playground powered by e8vm*

We'll compare these two types of posts to determine the following:

- Do Ask HN or Show HN receive more comments on average?
- Do posts created at a certain time receive more comments on average?

Let's start by importing the libraries we need and reading the data set into a list of lists.


In [1]:
from csv import reader
open_file = open("data/hacker_news.csv")
read_file = reader(open_file)
dataset = list(read_file)
headers = dataset[0]
dataset = dataset[1:]
print(headers)
print(dataset[1:6])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
[['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12'], ['10482257', 'Title II kills investment? Comcast and other ISPs are now spending more', 'http://arstechnica.com/business/2015/10/comcast-and-othe

Now let's seperate ask post and show posts

In [2]:
ask_posts = []
show_posts = []
other_posts = []

for row in dataset:
    title = row[1]
    title = title.lower()
    if title.startswith('ask hn'):
        ask_posts.append(row)
    elif title.startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
print("*** ASK HN ***")
print(ask_posts[1:4])
print("\n*** SHOW HN ***")
print(show_posts[1:4])

*** ASK HN ***
[['10610020', 'Ask HN: Am I the only one outraged by Twitter shutting down share counts?', '', '28', '29', 'tkfx', '11/22/2015 13:43'], ['11610310', 'Ask HN: Aby recent changes to CSS that broke mobile?', '', '1', '1', 'polskibus', '5/2/2016 10:14'], ['12210105', 'Ask HN: Looking for Employee #3 How do I do it?', '', '1', '3', 'sph130', '8/2/2016 14:20']]

*** SHOW HN ***
[['10646440', 'Show HN: Something pointless I made', 'http://dn.ht/picklecat/', '747', '102', 'dhotson', '11/29/2015 22:46'], ['11590768', 'Show HN: Shanhu.io, a programming playground powered by e8vm', 'https://shanhu.io', '1', '1', 'h8liu', '4/28/2016 18:05'], ['12178806', 'Show HN: Webscope  Easy way for web developers to communicate with Clients', 'http://webscopeapp.com', '3', '3', 'fastbrick', '7/28/2016 7:11']]


Since now we can start working on the first question: what posts receive more comments on average.

To answer the question, we will build the function that takes the list of posts and index of comments as the input and outputs average number of comments. 

In [3]:
def avg_comments(posts, index):
    total = 0
    for row in posts:
        comments = int(row[index]) #4
        total += comments
    avg_comments = total/len(posts)
    return avg_comments

Let's print average numbers of comments of each type and compare those.

In [4]:
ask_avg_comments = avg_comments(ask_posts, 4)
print("ASK HN average number of comments:", ask_avg_comments)
show_avg_comments = avg_comments(show_posts, 4)
print("SHOW HN average number of comments:", show_avg_comments)

ASK HN average number of comments: 14.038417431192661
SHOW HN average number of comments: 10.31669535283993


As we can see, the number of comments of ASK HN type is higher than SHOW HN

Let's answer now the second question - Do posts created at a certain time receive more comments on average?

To do that we will need to import another library - datetime and create two dictionaries of comments by hour and count of posts by hour where we will store hours as keys and number of comments/posts as values.

In [5]:
import datetime as dt

result_list = []
for row in ask_posts:
    created_at = row[6]
    comments = row[4]
    result_list.append([created_at, comments])
    
count_by_hour = {}
comments_by_hour = {}

for row in result_list:
    create_date = dt.datetime.strptime(row[0], "%m/%d/%Y %H:%M")
    create_date = create_date + dt.timedelta(hours = 5) # EST time +5 hours = BST time
    comments = int(row[1])
    create_hour = create_date.strftime("%H")
    
    if create_hour in count_by_hour:
        count_by_hour[create_hour] += 1
        comments_by_hour[create_hour] += comments
    else:
        count_by_hour[create_hour] = 1
        comments_by_hour[create_hour] = comments

Now we have two dictionaries - count_by_hour and comments_by_our. Next, we can iterate over one of the dictionaries and get the average number of comment per hour by dividing number of comments per hour by number of posts per hour.

In [6]:
avg_by_hour_list = []
for item in count_by_hour:
    avg_by_hour = int(comments_by_hour[item])/int(count_by_hour[item])
    avg_by_hour_list.append([item, avg_by_hour])
print(avg_by_hour_list)

[['14', 5.5777777777777775], ['18', 14.741176470588234], ['15', 13.440677966101696], ['19', 13.233644859813085], ['21', 16.796296296296298], ['04', 7.985294117647059], ['17', 9.41095890410959], ['22', 11.46], ['20', 38.5948275862069], ['02', 16.009174311926607], ['01', 21.525], ['07', 23.810344827586206], ['23', 13.20183486238532], ['08', 7.796296296296297], ['10', 10.08695652173913], ['00', 10.8], ['06', 11.383333333333333], ['03', 6.746478873239437], ['13', 10.25], ['09', 7.170212765957447], ['05', 8.127272727272727], ['11', 9.022727272727273], ['12', 7.852941176470588], ['16', 11.051724137931034]]


Here is our final output - the list of lists of average comments by hour. It seems to be difficult to analyze data in ths format. 

Let's print top 5 hours by average comments

In [7]:
swap_avg_by_hour_list = []
for row in avg_by_hour_list:
    swap_avg_by_hour_list.append([row[1], row[0]])
    
sorted_swap_list = sorted(swap_avg_by_hour_list, reverse = True)

print("*** TOP 5 Hours for Ask Posts Comments")

for row in sorted_swap_list[0:5]:
    print("{}:00: {:.2f} average comments per post".format(row[1], row[0]))

*** TOP 5 Hours for Ask Posts Comments
20:00: 38.59 average comments per post
07:00: 23.81 average comments per post
01:00: 21.52 average comments per post
21:00: 16.80 average comments per post
02:00: 16.01 average comments per post


# Conclusion

During this project we have answered two questions - what type of posts have more comments on average and what are top 5 hours for ask posts comments. 